In [1]:
import terragon
import geopandas as gpd
import json

In [2]:
with open("/localhome/hoeh_pa/Projects/terragon/demo_files/credentials.json") as file:
    credentials = json.load(file)

cdse = terragon.init('cdse', credentials)

In [3]:
collections = cdse.retrieve_collections()
print(collections)

['COP-DEM', 'S2GLC', 'TERRAAQUA', 'SENTINEL-3', 'SENTINEL-5P', 'SENTINEL-1-RTC', 'SENTINEL-1', 'SMOS', 'LANDSAT-7', 'CCM', 'LANDSAT-5', 'LANDSAT-8', 'ENVISAT', 'SENTINEL-6', 'GLOBAL-MOSAICS', 'SENTINEL-2']


In [ ]:
arguments = dict(
    shp=gpd.read_file("/localhome/hoeh_pa/Projects/terragon/demo_files/data/TUM_OTN.shp.zip"), 
    resolution=10,
    collection='SENTINEL-2', 
    start_date='2021-06-01', 
    end_date='2021-06-30', 
    download_folder='/localhome/hoeh_pa/Projects/terragon/demo_files/data/downloads', 
    num_workers=4
    )

In [5]:
items = cdse.search(**arguments)

In [6]:
from datetime import datetime
from pystac import ItemCollection

def filter_unique_items(items, preferred_level, tile_id, max_cloud_cover=50):
    def extract(item, key, default=None):
        # Extracts values from properties or defaults if not found
        return item.properties.get(key, default)

    seen = {}
    for item in items:
        if extract(item, 'tileId') != tile_id:
            continue  # Skip items that don't match the specified tileId

        date = datetime.strptime(item.id.split('_')[2][:8], '%Y%m%d').date()
        level, cloud_cover = extract(item, 'productType'), extract(item, 'cloudCover', 100)
        key = (date, tile_id)

        if key not in seen or \
           (level == preferred_level and extract(seen[key], 'productType') != preferred_level) or \
           (level == extract(seen[key], 'productType') and cloud_cover < extract(seen[key], 'cloudCover', 100)):

            if cloud_cover <= max_cloud_cover:
                seen[key] = item

    return ItemCollection(seen.values())

# Example usage:
items_filtered = filter_unique_items(items, preferred_level='S2MSI2A', tile_id='32UPU', max_cloud_cover=50)
items_filtered


In [7]:
item_dirs = cdse.download(items_filtered, create_minicube=False, delete_zip=True)
#cube = tg.download(selected_items, create_minicube=True, delete_zip=True)

Downloaded item: /localhome/hoeh_pa/Projects/terragon/demo_files/data/downloads/S2A_MSIL2A_20210602T101031_N0300_R022_T32UPU_20210602T171307.zip
Downloaded item: /localhome/hoeh_pa/Projects/terragon/demo_files/data/downloads/S2B_MSIL2A_20210720T101559_N0301_R065_T32UPU_20210720T132207.zip
Downloaded item: /localhome/hoeh_pa/Projects/terragon/demo_files/data/downloads/S2A_MSIL2A_20210702T101031_N0301_R022_T32UPU_20210702T121353.zip
Downloaded item: /localhome/hoeh_pa/Projects/terragon/demo_files/data/downloads/S2B_MSIL2A_20210610T101559_N0300_R065_T32UPU_20210610T132739.zip
Access token expired. Refreshing token and retrying...
Access token expired. Refreshing token and retrying...
Access token expired. Refreshing token and retrying...
Access token expired. Refreshing token and retrying...


In [13]:
ds = tg.build_minicube(arguments['download_folder'])

In [27]:
ds = build_minicube(output_dir='/dss/dsshome1/0F/di35viw/terragon/demo_files/downloads', shp='/dss/dsshome1/0F/di35viw/terragon/demo_files/data/TUM_OTN.shp.zip', target_res=10, num_workers=4)

In [34]:
ds.rio.bounds()

(696040.0, 5323140.0, 701060.0, 5327460.0)

In [35]:
gdf = gpd.read_file('/dss/dsshome1/0F/di35viw/terragon/demo_files/data/TUM_OTN.shp.zip')

In [39]:
gdf = gdf.to_crs(ds.rio.crs)
gdf.total_bounds

array([ 696044.56842919, 5323147.98448889,  701053.32696745,
       5327459.48676893])